<a href="https://colab.research.google.com/github/Jgil20/Account/blob/master/Copy_of_artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [3]:
import pandas as pd
import numpy as np 
import tensorflow as tf 

In [4]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [5]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:, 3:-1].values
exited = dataset.iloc[:,-1].values

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
print(exited)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [8]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X[:,2] = labelencoder.fit_transform(X[:,2])

In [9]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
columntransformer = ColumnTransformer(transformers=[('encoder',OneHotEncoder(), [1])],remainder='passthrough' )
X = np.array(columntransformer.fit_transform(X))

In [12]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test,exited_train,_exited_test = train_test_split(X,exited,test_size=0.2,random_state=0) 

### Feature Scaling

In [14]:
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
X_train = standardscaler.fit_transform(X_train)
X_test = standardscaler.fit_transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [15]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [16]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

### Adding the second hidden layer

In [17]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

### Adding the output layer

In [18]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [19]:
ann.compile(optimizer ='adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'] )

### Training the ANN on the Training set

In [22]:
ann.fit(X_train,exited_train, batch_size = 32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5360 - accuracy: 0.7477
Epoch 2/100
250/250 [==============================] - 0s 958us/step - loss: 0.4469 - accuracy: 0.8008
Epoch 3/100
250/250 [==============================] - 0s 944us/step - loss: 0.4359 - accuracy: 0.8045
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4294 - accuracy: 0.8083
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.8123
Epoch 6/100
250/250 [==============================] - 0s 996us/step - loss: 0.4179 - accuracy: 0.8202
Epoch 7/100
250/250 [==============================] - 0s 991us/step - loss: 0.4115 - accuracy: 0.8244
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4065 - accuracy: 0.8260
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4022 - accuracy: 0.8264
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3982 

## Part 4 - Making the predictions and evaluating the model

### Predicting the results of a single observation


**HomeWork**

Use our ANN model to predict if the customer with the following informationd will leave the bank 

Geography: France 

Credit Score:600

Gender Male

Age: 40 years old 

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer 

**Solution**

In [24]:
print(ann.predict(standardscaler.transform([[1,0,0, 600, 1,40,3,60000,2,1,1,50000]])) > 0.5)

[[False]]


Therefore,our ANN model predicts that this customer stays in the bank!

**Important note 1** Notice how the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs.And putting our values into a double pair of square brackets makes the input excaxtly a 2D array.

**Import note 2:** Notice also that the "France" country was not input as a string in the last column but as "1,0,0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X,"France" was encoded as "1,0,0". And be careful to include these values in the first three columns because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [30]:
exited_pred = ann.predict(X_test)
exited_pred = (exited_pred > 0.5)
print(np.concatenate((exited_pred.reshape(len(exited_pred),1), _exited_test.reshape(len(_exited_test),1)),1))


[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusionmatrix = confusion_matrix(_exited_test,exited_pred)
print(confusionmatrix)
accuracy_score(_exited_test,exited_pred)


[[1536   59]
 [ 216  189]]


0.8625